In [1]:
import solana
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana.transaction import TransactionInstruction, Transaction, AccountMeta
import argparse
from solana.publickey import PublicKey
from solana.blockhash import Blockhash
from solana.system_program import *
import base58, base64
from solana.sysvar import SYSVAR_RENT_PUBKEY
from solana.system_program import SYS_PROGRAM_ID
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.client import Token
import time

In [2]:
client = Client("https://api.devnet.solana.com")
ID = "Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g"
#ID = "A4vEBwVMoEZ8j4gthtMnm3MLgKtZoSrw7kYGq4KotE2Q"#"A4vEBwVMoEZ8j4gthtMnm3MLgKtZoSrw7kYGq4KotE2Q"
program_id=base58.b58decode(ID)

In [3]:
k = [239,114,243,183,37,213,148,161,13,172,25,86,143,240,175,157,215,108,156,181,161,230,0,40,228,255,23,209,52,254,120,234,234,209,190,175,172,89,1,135,75,124,43,221,38,148,22,210,210,175,41,10,176,56,96,41,166,103,76,50,58,78,15,74]
God = Keypair.from_secret_key(bytes(k))

In [4]:
God.public_key

GoduExX7V8SpBDTyf2pRukQgEFV8dXJjpaRuNRQAbfhb

In [5]:
Alice = Keypair()
Bob = Keypair()

In [6]:
client.request_airdrop(Alice.public_key, 1000000000)
time.sleep(15)
client.request_airdrop(Bob.public_key, 1000000000)

{'jsonrpc': '2.0',
 'result': '3ENTe1jMfd4H7Z2JDPFrHcPXnrhLpE1GLowr8f838BvSYPCsSUobC6qnsjVQ9gQrBXxMj7tde7V314ZxHLAXD8a',
 'id': 2}

In [7]:
X_mint = Token.create_mint(client, God, God.public_key, 9, TOKEN_PROGRAM_ID)
Y_mint = Token.create_mint(client, God, God.public_key, 9, TOKEN_PROGRAM_ID)

In [8]:
token_program_pk = TOKEN_PROGRAM_ID
x_seed = bytes(X_mint.pubkey)
y_seed = bytes(Y_mint.pubkey)
alice_seed = bytes(Alice.public_key)
bob_seed = bytes(Bob.public_key)
X_vault_pk, _ = PublicKey.find_program_address([x_seed, alice_seed, bob_seed], program_id)
Y_vault_pk, _ = PublicKey.find_program_address([y_seed, alice_seed, bob_seed], program_id)
x_vault_seed = bytes(X_vault_pk)
y_vault_seed = bytes(Y_vault_pk)
escrow_pk, _ = PublicKey.find_program_address([x_vault_seed, y_vault_seed, alice_seed, bob_seed], program_id)

In [9]:
# DEPOSIT
# create ATAs
X_a = X_mint.create_associated_token_account(Alice.public_key)
Y_a = Y_mint.create_associated_token_account(Alice.public_key)
X_b = X_mint.create_associated_token_account(Bob.public_key)
Y_b = Y_mint.create_associated_token_account(Bob.public_key)

In [10]:
X_mint.mint_to(X_a, God, 500)
#X_mint.mint_to(X_b, God, 500)
#Y_mint.mint_to(Y_a, God, 500)
Y_mint.mint_to(Y_b, God, 500)

{'jsonrpc': '2.0',
 'result': '63sb7Z4fYUj3EpVex5eFkprtk8h1EH3dJFm86rcAyouwhUeyWa97vqqQPcm7TwojHsGz8M97TJMptn44zTxsFU3X',
 'id': 43}

## Manual

In [49]:
account_metas = [
AccountMeta(Alice.public_key, True, False),
AccountMeta(Bob.public_key, False, False),
AccountMeta(X_mint.pubkey, False, False),
AccountMeta(Y_mint.pubkey, False, False),
AccountMeta(X_vault_pk, False, True),
AccountMeta(Y_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(SYS_PROGRAM_ID, False, False),
AccountMeta(SYSVAR_RENT_PUBKEY, False, False),
]

In [50]:
idx = 0
size_A = 10
size_B = 1
instruction_data = idx.to_bytes(1,"little") + size_A.to_bytes(8,"little") + size_B.to_bytes(8,"little")

In [51]:
instruction = TransactionInstruction(data=instruction_data,program_id=ID,keys=account_metas)
tix = Transaction().add(instruction)

In [52]:
# initialization
y = client.send_transaction(tix, Alice)
print(y)

{'jsonrpc': '2.0', 'result': '5eog7kkszmd1Ukz4VYRux9TxXNP4doRzqTit1UFCwUZZ2PkbPQLijhL2sSRuwtr92xTq242gRGgJBFjyBNpi7yXz', 'id': 108}


In [53]:
deposit_metas = [
AccountMeta(Alice.public_key, True, False),
AccountMeta(X_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(X_a, False, True),
AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
]

In [17]:
idx = 1
amount_a = 10
deposit_instruction_data = idx.to_bytes(1,"little") + amount_a.to_bytes(8,"little")

In [18]:
instruction = TransactionInstruction(data=deposit_instruction_data,program_id=ID,keys=deposit_metas)
tix = Transaction().add(instruction)

In [ ]:
y = client.send_transaction(tix, Alice)
print(y)

In [40]:
# WITHDRAW
withdraw_metas = [
AccountMeta(Alice.public_key, False, False),
AccountMeta(X_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(X_a, False, True),
AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
]

In [34]:
idx = 2
amount_a = 10
withdraw_instruction_data = idx.to_bytes(1,"little") + amount_a.to_bytes(8,"little")

In [41]:
instruction = TransactionInstruction(data=withdraw_instruction_data,program_id=ID,keys=withdraw_metas)
tix = Transaction().add(instruction)

In [ ]:
y = client.send_transaction(tix, Alice)

## TESTS

In [11]:
def initialize(size_A=10, size_B=1):
    account_metas = [
    AccountMeta(Alice.public_key, True, False),
    AccountMeta(Bob.public_key, False, False),
    AccountMeta(X_mint.pubkey, False, False),
    AccountMeta(Y_mint.pubkey, False, False),
    AccountMeta(X_vault_pk, False, True),
    AccountMeta(Y_vault_pk, False, True),
    AccountMeta(escrow_pk, False, True),
    AccountMeta(TOKEN_PROGRAM_ID, False, False),
    AccountMeta(SYS_PROGRAM_ID, False, False),
    AccountMeta(SYSVAR_RENT_PUBKEY, False, False),
    ]

    idx = 0
    instruction_data = idx.to_bytes(1,"little") + size_A.to_bytes(8,"little") + size_B.to_bytes(8,"little")
    
    instruction = TransactionInstruction(data=instruction_data,program_id=ID,keys=account_metas)
    
    return instruction

In [12]:
def deposit_alice(amount=10):
    deposit_metas = [
    AccountMeta(Alice.public_key, True, False),
    AccountMeta(X_vault_pk, False, True),
    AccountMeta(escrow_pk, False, True),
    AccountMeta(TOKEN_PROGRAM_ID, False, False),
    AccountMeta(X_a, False, True),
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    ]
    
    idx = 1
    deposit_instruction_data = idx.to_bytes(1,"little") + amount.to_bytes(8,"little")
    
    instruction = TransactionInstruction(data=deposit_instruction_data,program_id=ID,keys=deposit_metas)
    
    return instruction

In [13]:
def deposit_bob(amount=1):
    deposit_metas = [
    AccountMeta(Bob.public_key, True, False),
    AccountMeta(Y_vault_pk, False, True),
    AccountMeta(escrow_pk, False, True),
    AccountMeta(TOKEN_PROGRAM_ID, False, False),
    AccountMeta(Y_b, False, True),
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    ]
    
    idx = 1
    deposit_instruction_data = idx.to_bytes(1,"little") + amount.to_bytes(8,"little")
    
    instruction = TransactionInstruction(data=deposit_instruction_data,program_id=ID,keys=deposit_metas)
    
    return instruction

In [14]:
def withdraw_alice_x(amount=10):
    withdraw_metas = [
    AccountMeta(Alice.public_key, False, False),
    AccountMeta(X_vault_pk, False, True),
    AccountMeta(escrow_pk, False, True),
    AccountMeta(TOKEN_PROGRAM_ID, False, False),
    AccountMeta(X_a, False, True),
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    ]
    
    idx = 2
    withdraw_instruction_data = idx.to_bytes(1,"little") + amount.to_bytes(8,"little")

    instruction = TransactionInstruction(data=withdraw_instruction_data,program_id=ID,keys=withdraw_metas)
    
    return instruction

In [15]:
def withdraw_alice_y(amount=1):
    withdraw_metas = [
    AccountMeta(Alice.public_key, False, False),
    AccountMeta(Y_vault_pk, False, True),
    AccountMeta(escrow_pk, False, True),
    AccountMeta(TOKEN_PROGRAM_ID, False, False),
    AccountMeta(Y_a, False, True),
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    ]
    
    idx = 2
    withdraw_instruction_data = idx.to_bytes(1,"little") + amount.to_bytes(8,"little")

    instruction = TransactionInstruction(data=withdraw_instruction_data,program_id=ID,keys=withdraw_metas)
    
    return instruction

In [16]:
def withdraw_bob_x(amount=10):
    withdraw_metas = [
    AccountMeta(Bob.public_key, False, False),
    AccountMeta(X_vault_pk, False, True),
    AccountMeta(escrow_pk, False, True),
    AccountMeta(TOKEN_PROGRAM_ID, False, False),
    AccountMeta(X_b, False, True),
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    ]
    
    idx = 2
    withdraw_instruction_data = idx.to_bytes(1,"little") + amount.to_bytes(8,"little")

    instruction = TransactionInstruction(data=withdraw_instruction_data,program_id=ID,keys=withdraw_metas)
    
    return instruction

In [17]:
def withdraw_bob_y(amount=1):
    withdraw_metas = [
    AccountMeta(Bob.public_key, False, False),
    AccountMeta(Y_vault_pk, False, True),
    AccountMeta(escrow_pk, False, True),
    AccountMeta(TOKEN_PROGRAM_ID, False, False),
    AccountMeta(Y_b, False, True),
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    ]
    
    idx = 2
    withdraw_instruction_data = idx.to_bytes(1,"little") + amount.to_bytes(8,"little")

    instruction = TransactionInstruction(data=withdraw_instruction_data,program_id=ID,keys=withdraw_metas)
    
    return instruction

### Test 1: Alice deposits, Bob deposits, Alice withdraws, Bob withdraws

In [19]:
tix = Transaction()
instructions = [initialize(10,1),deposit_alice(10),deposit_bob(1),withdraw_alice_y(1),withdraw_bob_x(10)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice,Bob)

### Test 2: Alice deposits, Bob deposits, Bob withdraws, Alice withdraws

In [20]:
tix = Transaction()
instructions = [initialize(10,1),deposit_alice(10),deposit_bob(1),withdraw_bob_x(10),withdraw_alice_y(1)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice,Bob)

### Test 3: Alice deposits, Alice withdraws

In [22]:
tix = Transaction()
instructions = [initialize(10,1),deposit_alice(10),withdraw_alice_x(10)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice)

### Test 4: Bob deposits, Bob withdraws

In [23]:
tix = Transaction()
instructions = [initialize(10,1),deposit_bob(1),withdraw_bob_y(1)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix,Bob)

### Test 5: Alice deposits, Alice withdraws from Y (SHOULD FAIL)

In [24]:
tix = Transaction()
instructions = [initialize(10,1),deposit_alice(10),withdraw_alice_y(1)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 2: custom program error: 0x3', 'data': {'accounts': None, 'err': {'InstructionError': [2, {'Custom': 3}]}, 'logs': ['Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: InitEscrow', 'Program log: 0', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 4561 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g success', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: Deposit', 'Program log: Getting escrow data', 'Program log: ATA KEY AND BUMP BELOW', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [2]', 'Program log: Instruction: Transfer', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 3121 of 188270 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA success', 'Program log: Done with invoke', 'Program log: Alice deposi

### Test 6: Alice deposits, Alice withdraws (more than can, SHOULD FAIL)

In [25]:
tix = Transaction()
instructions = [initialize(10,1),deposit_alice(10),withdraw_alice_x(11)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 2: custom program error: 0x1', 'data': {'accounts': None, 'err': {'InstructionError': [2, {'Custom': 1}]}, 'logs': ['Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: InitEscrow', 'Program log: 0', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 4561 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g success', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: Deposit', 'Program log: Getting escrow data', 'Program log: ATA KEY AND BUMP BELOW', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [2]', 'Program log: Instruction: Transfer', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 3121 of 188270 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA success', 'Program log: Done with invoke', 'Program log: Alice deposi

### Test 7: Alice deposits (less than allowed) (SHOULD FAIL)

In [26]:
tix = Transaction()
instructions = [initialize(10,1),deposit_alice(1)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 1: custom program error: 0x1', 'data': {'accounts': None, 'err': {'InstructionError': [1, {'Custom': 1}]}, 'logs': ['Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: InitEscrow', 'Program log: 0', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 4561 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g success', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: Deposit', 'Program log: Getting escrow data', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 3459 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g failed: custom program error: 0x1']}}

### Test 8: Alice deposits (more than allowed) (SHOULD FAIL)

In [27]:
tix = Transaction()
instructions = [initialize(10,1),deposit_alice(11)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 1: custom program error: 0x1', 'data': {'accounts': None, 'err': {'InstructionError': [1, {'Custom': 1}]}, 'logs': ['Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: InitEscrow', 'Program log: 0', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 4561 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g success', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: Deposit', 'Program log: Getting escrow data', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 3459 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g failed: custom program error: 0x1']}}

### Test 9: Alice deposits, Bob deposits, Bob withdraws, Bob withdraws (SHOULD FAIL)

In [30]:
tix = Transaction()
instructions = [initialize(10,1),deposit_alice(10),deposit_bob(1),withdraw_bob_x(10),withdraw_bob_x(10)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice, Bob)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 4: custom program error: 0x7', 'data': {'accounts': None, 'err': {'InstructionError': [4, {'Custom': 7}]}, 'logs': ['Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: InitEscrow', 'Program log: 0', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 4559 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g success', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: Deposit', 'Program log: Getting escrow data', 'Program log: ATA KEY AND BUMP BELOW', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [2]', 'Program log: Instruction: Transfer', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 3121 of 188270 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA success', 'Program log: Done with invoke', 'Program log: Alice deposi

### Test 10: Alice deposits, Alice deposits (SHOULD FAIL)

In [31]:
tix = Transaction()
instructions = [initialize(10,1),deposit_alice(10),deposit_alice(10)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 2: custom program error: 0x4', 'data': {'accounts': None, 'err': {'InstructionError': [2, {'Custom': 4}]}, 'logs': ['Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: InitEscrow', 'Program log: 0', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 4559 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g success', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: Deposit', 'Program log: Getting escrow data', 'Program log: ATA KEY AND BUMP BELOW', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [2]', 'Program log: Instruction: Transfer', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 3121 of 188270 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA success', 'Program log: Done with invoke', 'Program log: Alice deposi

### Test 11: Alice deposits, Bob deposits, Alice withdraws from X (SHOULD FAIL)

In [32]:
tix = Transaction()
instructions = [initialize(10,1),deposit_alice(10),deposit_bob(1),withdraw_alice_x(10)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice, Bob)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 3: custom program error: 0x5', 'data': {'accounts': None, 'err': {'InstructionError': [3, {'Custom': 5}]}, 'logs': ['Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: InitEscrow', 'Program log: 0', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 4559 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g success', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: Deposit', 'Program log: Getting escrow data', 'Program log: ATA KEY AND BUMP BELOW', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [2]', 'Program log: Instruction: Transfer', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 3121 of 188270 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA success', 'Program log: Done with invoke', 'Program log: Alice deposi

### Test 12: Alice withdraws (SHOULD FAIL)

In [33]:
tix = Transaction()
instructions = [initialize(10,1),withdraw_alice_x(10)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 1: custom program error: 0x6', 'data': {'accounts': None, 'err': {'InstructionError': [1, {'Custom': 6}]}, 'logs': ['Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: InitEscrow', 'Program log: 0', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 4561 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g success', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: Withdraw', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 3288 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g failed: custom program error: 0x6']}}

### Test 13: Alice deposits, Bob withdraws (SHOULD FAIL)

In [34]:
tix = Transaction()
instructions = [initialize(10,1),deposit_alice(10),withdraw_bob_x(10)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice, Bob)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 2: custom program error: 0x2', 'data': {'accounts': None, 'err': {'InstructionError': [2, {'Custom': 2}]}, 'logs': ['Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: InitEscrow', 'Program log: 0', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 4559 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g success', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: Deposit', 'Program log: Getting escrow data', 'Program log: ATA KEY AND BUMP BELOW', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [2]', 'Program log: Instruction: Transfer', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 3121 of 188270 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA success', 'Program log: Done with invoke', 'Program log: Alice deposi

### Test 13: Someone else tries to withdraw (SHOULD FAIL)

In [48]:
def withdraw_crassus_y(amount=1):
    withdraw_metas = [
    AccountMeta(Keypair().public_key, False, False),
    AccountMeta(Y_vault_pk, False, True),
    AccountMeta(escrow_pk, False, True),
    AccountMeta(TOKEN_PROGRAM_ID, False, False),
    AccountMeta(Y_b, False, True),
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    ]
    
    idx = 2
    withdraw_instruction_data = idx.to_bytes(1,"little") + amount.to_bytes(8,"little")

    instruction = TransactionInstruction(data=withdraw_instruction_data,program_id=ID,keys=withdraw_metas)
    
    return instruction

In [49]:
tix = Transaction()
instructions = [initialize(10,1),deposit_alice(10),deposit_bob(1),withdraw_bob_x(10),withdraw_crassus_y(1)]
for instruction in instructions:
    tix.add(instruction)
y = client.send_transaction(tix, Alice, Bob)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 4: custom program error: 0x0', 'data': {'accounts': None, 'err': {'InstructionError': [4, {'Custom': 0}]}, 'logs': ['Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: InitEscrow', 'Program log: 0', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g consumed 4559 of 200000 compute units', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g success', 'Program Cf8D7jF1mxtn1uX7kBWDUJZUQ5U3ewreM8JaxJDda77g invoke [1]', 'Program log: Instruction: Deposit', 'Program log: Getting escrow data', 'Program log: ATA KEY AND BUMP BELOW', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [2]', 'Program log: Instruction: Transfer', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 3121 of 188270 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA success', 'Program log: Done with invoke', 'Program log: Alice deposi

In [45]:
# Alice, Bob both deposit and then withdraw
tix = Transaction()
#X_mint.mint_to(X_a, God, 500)
#Y_mint.mint_to(Y_b, God, 500)

# alice deposit
deposit_metas = [
AccountMeta(Alice.public_key, True, False),
AccountMeta(X_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(X_a, False, True),
AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
]
idx = 1
amount_a = 10
deposit_instruction_data = idx.to_bytes(1,"little") + amount_a.to_bytes(8,"little")
instruction = TransactionInstruction(data=deposit_instruction_data,program_id=ID,keys=deposit_metas)
tix.add(instruction)
y = client.send_transaction(tix, Alice)
print(y)

In [47]:
# bob deposit
tix = Transaction()
deposit_metas = [
AccountMeta(Bob.public_key, True, False),
AccountMeta(Y_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(Y_b, False, True),
AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
]
idx = 1
amount_b = 1
deposit_instruction_data = idx.to_bytes(1,"little") + amount_b.to_bytes(8,"little")
instruction = TransactionInstruction(data=deposit_instruction_data,program_id=ID,keys=deposit_metas)
tix.add(instruction)
y = client.send_transaction(tix, Bob)
print(y)

In [54]:
# alice withdraw
tix = Transaction()
withdraw_metas = [
AccountMeta(Alice.public_key, False, False),
AccountMeta(Y_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(Y_a, False, True),
AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
]
idx = 2
amount_b = 1
withdraw_instruction_data = idx.to_bytes(1,"little") + amount_b.to_bytes(8,"little")
instruction = TransactionInstruction(data=withdraw_instruction_data,program_id=ID,keys=withdraw_metas)
tix.add(instruction)
y = client.send_transaction(tix, Alice)
print(y)

In [56]:
# bob withdraw
tix = Transaction()
withdraw_metas = [
AccountMeta(Bob.public_key, False, False),
AccountMeta(X_vault_pk, False, True),
AccountMeta(escrow_pk, False, True),
AccountMeta(TOKEN_PROGRAM_ID, False, False),
AccountMeta(X_b, False, True),
AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
]
idx = 2
amount_a = 10
withdraw_instruction_data = idx.to_bytes(1,"little") + amount_a.to_bytes(8,"little")
instruction = TransactionInstruction(data=withdraw_instruction_data,program_id=ID,keys=withdraw_metas)
tix.add(instruction)

# send to client
y = client.send_transaction(tix, Bob)
print(y)